In [5]:
using XLSX, DataFrames, TimeSeries
using StatsBase
using Plots, StatsPlots
using LinearAlgebra
using GLM
using FredData
include("utils.jl")


print_percentiles (generic function with 1 method)

In [54]:
data = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Macro_Data"; infer_eltypes=true))
data = TimeArray(data,timestamp=:Date)


210×12 TimeArray{Any, 2, Date, Matrix{Any}} 1971-10-01 to 2024-01-01
┌────────────┬───────────────┬─────────────────┬─────────────────┬──────────────
│            │ NGDPRSAXDCPLQ │ POLCPIALLQINMEI │ IRLTLT01PLM156N │ IR3TIB01PLQ ⋯
├────────────┼───────────────┼─────────────────┼─────────────────┼──────────────
│ 1971-10-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-01-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-04-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-07-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-10-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-01-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-04-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-07-01 │       missing │         missing │         missing │         mis ⋯
│     ⋮      │       ⋮       │        ⋮ 

In [55]:
short_data = from(data[:,[:NGDPRSAXDCPLQ, :POLCPIALLQINMEI, :IRLTLT01PLM156N, :IR3TIB01PLQ156N, :NGDPRSAXDCUSQ, :USACPIALLMINMEI, :IRLTLT01USQ156N, :IR3TIB01USQ156N, :CLVMEURSCAB1GQEA19, :CP0000EZ19M086NEST, :IRLTLT01EZM156N, :IR3TIB01EZQ156N]],Date(2001,1,1))

93×12 TimeArray{Any, 2, Date, Matrix{Any}} 2001-01-01 to 2024-01-01
┌────────────┬───────────────┬─────────────────┬─────────────────┬──────────────
│            │ NGDPRSAXDCPLQ │ POLCPIALLQINMEI │ IRLTLT01PLM156N │ IR3TIB01PLQ ⋯
├────────────┼───────────────┼─────────────────┼─────────────────┼──────────────
│ 2001-01-01 │     2.49527e5 │          73.028 │         10.4267 │         18. ⋯
│ 2001-04-01 │     2.48736e5 │         74.3877 │         11.1767 │             ⋯
│ 2001-07-01 │      249960.0 │         74.3408 │         11.7133 │         15. ⋯
│ 2001-10-01 │     2.50033e5 │         74.7863 │            9.41 │         12. ⋯
│ 2002-01-01 │     2.51671e5 │         75.6303 │         8.30667 │             ⋯
│ 2002-04-01 │     2.53462e5 │         75.9116 │         7.92667 │         9.5 ⋯
│ 2002-07-01 │     2.55909e5 │         75.2083 │            7.21 │             ⋯
│ 2002-10-01 │     2.57161e5 │         75.4427 │            5.98 │         6.8 ⋯
│     ⋮      │       ⋮       │        ⋮  

In [56]:
returns = percentchange(short_data[:,[:NGDPRSAXDCPLQ, :POLCPIALLQINMEI, :NGDPRSAXDCUSQ, :USACPIALLMINMEI, :CLVMEURSCAB1GQEA19, :CP0000EZ19M086NEST]])

TypeError: TypeError: non-boolean (Missing) used in boolean context

In [57]:
ar = lm(@formula(CPI_PLN ~ GLM.lag(CPI_PLN,1)),cpi)

UndefVarError: UndefVarError: `cpi` not defined

In [13]:
inflation = percentchange(cpi,:log)


TypeError: TypeError: non-boolean (Missing) used in boolean context